# Solving Titanic Kaggle Challenge with a Neural Network

In [ ]:
!pip install wandb

In [ ]:
!wandb login APIKEY

In [ ]:
import torch
from torch import nn
import torch.optim as optim

import pandas as pd
import os
from torch.utils.data import DataLoader, TensorDataset
import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score

import wandb
import argparse

## Choose a model

In [ ]:
# Here we define our model
# It is a simple feedforward neural network

# Input Layer --> number of features from our dataset
# Hidden Layers
# Activation Function: ReLU (Rectified Linear Unit) --> this is for introducing non-linearity
# Output Layer: 1 neuron with a sigmoid activation function, representing the survival probability
class TitanicNN(nn.Module):
    def __init__(self, input_size):
        super(TitanicNN, self).__init__()

        # This is the first hidden layer with 128 neurons
        # It takes the input features and applies a linear transformation followed by the ReLU activation function
        self.fc1 = nn.Linear(input_size, 128)

        # Second hidden layer with 64 neurons. It takes the output of the first hidden layer and applies another
        # linear transformation followed by the ReLU activation function
        self.fc2 = nn.Linear(128, 64)

        # Output layer with 1 neuron. It takes the output of the second hidden layer and applies a linear transformation
        # followed by the sigmoid activation function.
        self.fc3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

#device = "cpu" # uncomment if you want to use "cpu", currently cpu is faster than cuda (maybe because the NN is very little)
print(f"Using {device} device")

number_of_featuers = 5
model = TitanicNN(number_of_featuers).to(device)

## Choose a loss function

For binary classification tasks, the most common choice of loss function is the binary cross-entropy loss (log loss).

In [ ]:
loss_function = nn.BCELoss()

In [ ]:
# loads just the training data
# returns two DataFrames, one with just the features and one with the labels
def load_titanic_train_preprocessed():
    train_preprocessed = pd.read_csv(os.path.join('data', 'train_preprocessed.csv'))
    train_preprocessed_features = train_preprocessed.drop('Survived', axis=1)
    train_preprocessed_label = train_preprocessed['Survived']
    return train_preprocessed_features, train_preprocessed_label

In [ ]:
# loads just the test data
# returns two DataFrames, one with just the features and one with the labels
def load_titanic_train_test_preprocessed():
    train_test_preprocessed = pd.read_csv(os.path.join('data', 'train_test_preprocessed.csv'))
    train_test_preprocessed_features = train_test_preprocessed.drop('Survived', axis=1)
    train_test_preprocessed_label = train_test_preprocessed['Survived']
    return train_test_preprocessed_features, train_test_preprocessed_label

In [13]:
for i in range(7, 12):  # 2^7 = 128, 2^12 = 4096
    # Initialize wandb
    # config hyperparameter (usually from command line) and let W&B know about them
    config = argparse.Namespace()
    config.learning_rate = 1e-3
    config.epochs = 10000
    config.batch_size = 2**i

    wandb.init(project="kaggle-titanic", config=vars(config))

    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    train_features, train_labels = load_titanic_train_preprocessed()

    # convert to PyTorch tensors
    features_tensors = torch.tensor(train_features.values, dtype=torch.float32)
    targets_tensors = torch.tensor(train_labels.values, dtype=torch.float32)

    # Create a TensorDataset
    dataset = TensorDataset(features_tensors, targets_tensors)

    # Define data loader
    # It contains both the features and the labels
    batch_size = config.batch_size
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    wandb.watch(model)
    model.train(mode=True)

    losses = []
    for epoch in range(config.epochs):
        for batch_inputs, batch_targets in data_loader:
            # batch_inputs --> my features
            # batch_targets --> my expected labels

            # Clear gradients from previous iteration
            optimizer.zero_grad()

            # now we make predictions from our model
            batch_outputs = model(batch_inputs.to(device))

            # and we calculate the loss by comparing the predictions against our expected labels
            # since batch_targets has shape [32] (1 row, 32 cols) and batch_outputs has [32,1]
            # (32 rows, 1 col), we do an unsqueeze here to change the shape of batch_targets
            loss = loss_function(batch_outputs, batch_targets.unsqueeze(1).to(device))
            losses.append(loss.item())

            # Backpropagate gradients
            loss.backward()

            # Update model parameters
            optimizer.step()

        wandb.log({'epoch': epoch+1, 'loss': loss.item()})

    model.eval()

    test_features, test_labels = load_titanic_train_test_preprocessed()

    # convert to PyTorch tensors
    test_features_tensors = torch.tensor(test_features.values, dtype=torch.float32)

    predicted_labels = model(test_features_tensors.to(device))

    # The predicted_labels contain values between 0 and 1.
    # We apply a threshold to make a binary classification.
    threshold = 0.5
    binary_predictions = [1 if prob >= threshold else 0 for prob in predicted_labels]

    wandb.log({"test_accuracy": f1_score(test_labels, binary_predictions, average="weighted")})
    wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▇█▅█▆▃▅▆▂▄▅▃▅▁▄▃▅▂▅▄▄▃▄▃▃▂▅▆▂▃▅▆▄▂▅▅▅▁▃▃
test_accuracy,▁
epoch,10000
loss,0.32612
test_accuracy,0.73119


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▄▆▃▇▄▅▅▅▄▇▅▄▄▄▂▅▄█▆▂▃█▄▄▃▃▃▁▄▃▆▆▄▃▅▆▃▇▂▄
test_accuracy,▁
epoch,10000
loss,0.2434
test_accuracy,0.75663


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▆▇▆▇▅▅▃▂▄▅▅▃▄▅▄▃▅▅▅▅▆▅▅▄█▅▄▅▆▄▃▄▅▆▅▆▅▅▂▁
test_accuracy,▁
epoch,10000
loss,0.28163
test_accuracy,0.74395


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
test_accuracy,▁
epoch,10000
loss,0.21418
test_accuracy,0.75144


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,███▇▇▇▇▆█▆▇▆▆▅▅▅▅▇▅▅▄▄▄▃▃▄▄▃▄▃▃▃▂▂▂▂▂▁▁▁
test_accuracy,▁
epoch,10000
loss,0.2102
test_accuracy,0.76832
